<a href="https://colab.research.google.com/github/QinLab/LLM-for-genetics/blob/main/mini_transformer_genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import pandas as pd
df = pd.read_csv("/content/who_train_transformer.csv")

In [147]:
len(df)

8556

In [144]:
characters = ['-', 'a', 'c', 'g', 'i', 'n', 't']
ntoi = {'-':0, 'a':1, 'c':2, 'g':3, 'i':4, 'n':5, 't': 6}
iton = {0:'-', 1:'a', 2:'c', 3:'g', 4:'i', 5:'n', 6:'t'}
encode = lambda s: [ntoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([iton[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("accgg--"))
print(decode(encode("accgg--")))

[1, 2, 2, 3, 3, 0, 0]
accgg--


In [145]:
k_mer = 6
ix = encoded_length - k_mer

all_seq_inputs = []
all_seq_targets = []

for seq_encode in x_data:
    seq_input = torch.stack([seq_encode[t:t+k_mer] for t in range(ix)])
    seq_target = torch.stack([seq_encode[t+1:t+k_mer+1] for t in range(ix)])

    all_seq_inputs.append(seq_input)
    all_seq_targets.append(seq_target)

all_seq_inputs = torch.cat(all_seq_inputs, dim=0)
all_seq_targets = torch.cat(all_seq_targets, dim=0)

In [146]:
len(all_seq_inputs), len(all_seq_targets)

(2988500, 2988500)

In [148]:
seq_input[18756], seq_target[18756]

(tensor([6., 6., 6., 1., 6., 3.]), tensor([6., 6., 1., 6., 3., 1.]))

In [149]:
n = int(0.9*len(all_seq_inputs))
train_data = all_seq_inputs[:n]
val_data = all_seq_inputs[n:]

In [150]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-1, (batch_size,))  # since in the line below, we have i+block_size, here we subtract block_size from len(data)
    x = torch.stack([data[i] for i in ix])
    y = torch.stack([data[i+1] for i in ix])
    return x, y

In [142]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.202247 M parameters
step 0: train loss 2.0106, val loss 2.0090
step 100: train loss 1.3582, val loss 1.3731
step 200: train loss 1.3409, val loss 1.3567
step 300: train loss 1.3124, val loss 1.3235
step 400: train loss 1.3156, val loss 1.3310
step 500: train loss 1.3153, val loss 1.3343
step 600: train loss 1.3068, val loss 1.3215
step 700: train loss 1.3129, val loss 1.3233
step 800: train loss 1.3198, val loss 1.3262
step 900: train loss 1.3088, val loss 1.3262
step 1000: train loss 1.3029, val loss 1.3231
step 1100: train loss 1.3065, val loss 1.3260
step 1200: train loss 1.3172, val loss 1.3233
step 1300: train loss 1.3216, val loss 1.3337
step 1400: train loss 1.3044, val loss 1.3197
step 1500: train loss 1.3056, val loss 1.3199
step 1600: train loss 1.3008, val loss 1.3160
step 1700: train loss 1.3033, val loss 1.3169
step 1800: train loss 1.3097, val loss 1.3167
step 1900: train loss 1.3073, val loss 1.3207
step 2000: train loss 1.3025, val loss 1.3225
step 2100: train loss 1.